In [1]:
import ultralytics
from ultralytics import YOLO
import PIL
import os
import shutil
import albumentations as A
import cv2
import os
import numpy as np
from collections import Counter, defaultdict
import csv
import supervision as sv

ultralytics.checks()

Ultralytics 8.3.61 🚀 Python-3.12.8 torch-2.5.1+cu124 CUDA:0 (NVIDIA A30, 24167MiB)
Setup complete ✅ (96 CPUs, 502.8 GB RAM, 0.4/10.0 GB disk)


## Notebook for running prediction and quantification of on malarial gametocyte stages.

## Classes
    -Early
    -stage 2
    -stage 3
    -stage 4
    -stage 5
    -debris 

Most classes will have good results with high confidence values, stage 4 is difficult and ambiguous but results should be around ~60% accurate
    
## Steps for usage
    1. Select environment from list by clicking select kernal in the top right. The environment is located in /vast/projects/BAC_shared/conda_envs/yolo_env
    2. Modify filepaths for Model weights and location of images to predict. Images must be in .jpg or .png format
    3. Run cells by selecting run all at the top or selecting individual cells and pressing shift+enter
    4. Csv counts and confidence values csv files will be output in the directory and will be replaced with every run. Make sure to download them each time after running counting


In [ ]:
# Load the pre-trained YOLO model
model = YOLO('/vast/scratch/users/mckay.m/malaria_classifier_katrina/runs/detect/train_2_11_full_images_size_640_best_so_far/weights/best.pt')

img_dir = '/vast/scratch/users/mckay.m/malaria_classifier_katrina/mal_dect_gametocyte_2-11/test/images'

# Run predictions on the test set images
results = model.predict(img_dir, save=True)

# Initialize data structures
all_stages = set()
csv_data = []
confidence_data = []

# Loop through results to find all unique stages
for result in results:
    # Extract the image name after the last "/"
    image_name = os.path.basename(result.path)
    
    # Get detected class indices and corresponding confidence scores
    class_indices = result.boxes.cls.int().tolist()
    confidences = result.boxes.conf.tolist()
    
    # Map class indices to stage names and create a list of tuples (stage, confidence)
    detected = [(model.names[class_idx], conf) for class_idx, conf in zip(class_indices, confidences)]
    
    # Add detected stages to the all_stages set
    all_stages.update(stage for stage, _ in detected)

# Convert the set of all stages to a sorted list
all_stages = sorted(all_stages)

# Totals for each stage
total_counts = Counter()

# Loop through results again to build rows for both CSVs
for result in results:
    # Extract the image name
    image_name = os.path.basename(result.path)
    
    # Get detected class indices and corresponding confidence scores
    class_indices = result.boxes.cls.int().tolist()
    confidences = result.boxes.conf.tolist()
    
    # Map class indices to stage names and create a list of tuples (stage, confidence)
    detected = [(model.names[class_idx], conf) for class_idx, conf in zip(class_indices, confidences)]
    
    # Count the occurrences of each detected stage
    stage_count = Counter(stage for stage, _ in detected)
    
    # Update totals for stage counts
    total_counts.update(stage_count)
    
    # Create a row for the detection counts
    count_row = [image_name]
    for stage in all_stages:
        count_row.append(stage_count.get(stage, 0))
    csv_data.append(count_row)
    
    # Create a row for confidence scores as a single column
    confidence_list = [f"{stage}:{round(conf, 2)}" for stage, conf in detected]
    confidence_row = [image_name, ", ".join(confidence_list)]
    confidence_data.append(confidence_row)

# Add totals to the counts CSV
total_row_counts = ["Total"]
for stage in all_stages:
    total_row_counts.append(total_counts.get(stage, 0))
csv_data.append(total_row_counts)

# Write the detection counts to the first CSV file
csv_file_counts = 'detection_results_counts.csv'
with open(csv_file_counts, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Image Name"] + all_stages)
    writer.writerows(csv_data)

print(f"Detection counts saved to {csv_file_counts}")

# Write the confidence scores to the second CSV file
csv_file_confidences = 'detection_results_confidences.csv'
with open(csv_file_confidences, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Image Name", "Detections with Confidence"])
    writer.writerows(confidence_data)

print(f"Confidence scores saved to {csv_file_confidences}")
